## 증권리포트 크롤링

- 매일경제증권센터 증권리포트 항목 크롤링
- 검색어, 검색할 maxPage 설정

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd

#증권리포트 제목, 내용, url을 분리, 저장하는 함수
def get_report_inf(url):
    #headers = {'Accept-Encoding': 'deflate'}
    #res = requests.get(url=url, headers=headers)
    res = requests.get(url=url)
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')#, from_encoding="euc-kr"
    title = soup.find('p', {'class':'tit'}).text
    text = soup.find('div', {'id':'article_main'}).text
    report_data = {}
    report_data['title'] = title.replace("\r"," ").replace("\n"," ").replace("\t"," ").strip()  # 기사 제목
    report_data['text'] = text.replace("\n", " ").replace("\r", " ").replace("<br>", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
    report_data['url'] = url  # 기사 링크
    
    return report_data

In [2]:
#증권리포트 크롤링 data를 반환하는 함수

def get_report_list(page=1  # 뉴스 리스트 페이지
                      ):
    
    
    url='https://www.thebell.co.kr/free/content/article.asp?page={page}&svccode=00'
    
    res = requests.get(url.format(page=page))
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')
    #print(soup)
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출    
    table = soup.find('div', {'class':'listBox'})
    alinks = table.findAll('a')
    
    report_list = []
    
    for alink in alinks:
        alink['href'] = 'https://www.thebell.co.kr/free/content/' + alink['href']
        report_url = alink['href']
        report_list.append(get_report_inf(report_url))
    
    return report_list

In [3]:
#증권리포트 크롤링 실행
articles = []

for num in range(1, 21):
    articles.extend(get_report_list(page=num))

print("Total articles: {:,}".format(len(articles)))
#print("="*20, "Article sample", "="*20)
#print(articles[0])

Total articles: 200


In [4]:
df = pd.DataFrame(articles)
df[['title']].head(10)

title
0  유니온투자, '사무실 중개' 알스퀘어에 130억 투자           프롭테크 1...
1  '패션공유 플랫폼' 클로젯셰어, 시리즈A 44억 유치           디쓰리·산은...
2  테스트웍스 "소외계층도 AI전문가로 성장"           윤석원 대표 "발달장애...
3  ㈜두산, 악조건 속 공모채 추진…완판행진 이을까           [발행사분석]계열...
4  한신공영, 240억 사모채 발행           1년6개월물 190억·2년물 50...
5  SK그룹, '루브리컨츠 상흔'에 바이오팜 결단 장고           계열사 공모 ...
6  '캐리언니' 캐리소프트, IPO 재도전 나선다           밸류에이션 등 눈높...
7  SK건설, 댐 붕괴 사고 후 첫 공모채…조달 본격화           최근 킥오프 ...
8  '수익 급감' 포스코건설, 수급 불안 불똥 맞나           [발행사분석]EB...
9  녹십자홀딩스, 사상 첫 공모채 발행 착수           3·5년물, 1000억 ...

## 기업리스트로 단어 Filtering
- 상장기업이름이 포함된 기사들로만 wordlist를 재구성한다

In [8]:
import mysql.connector 
from sqlalchemy import create_engine
db_host = '10.250.114.79'
db_name = 'mysql'
db_id = 'root'
db_pw = 'skcc'
cnx_str = 'mysql+mysqlconnector://' + db_id + ':' + db_pw + '@' + db_host + '/' + db_name + ''
engine = create_engine(cnx_str,echo=False)
with engine.connect() as con:
    rs = con.execute("SELECT * FROM co_list ")
    data = rs.fetchall()

print("DATA: ", data[0][1])

co_list = [] #기업리스트
#print(data)
for co_no,co_name in data:
    co_name = co_name.replace("\r", "")
    co_list.append(co_name)
    
#print(co_list)

DATA:  그린플러스


## Text preprocessing

- 형태소 분류작업 (정규화, 어근화작업도 수행)

In [9]:
from konlpy.tag import Okt
from collections import Counter
# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)


#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set([])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..
#stopwords.add(searchword)  #검색어 자체가 키워드에 포함되지 않게한다


for i, article in enumerate(articles):
    words = tagger.pos(article['text'], norm=True, stem=True) #정규화, 어근화도 진행
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    #print(tmp_words)
    co_words = [w for w in tmp_words if w in co_list]  #기업리스트로 필터링작업
    #print(co_words)on
    if len(co_words) > 0:  #기업리스트 단어들 따로 저장
        vocab.update(tmp_words)
        words_list.append((tmp_words, article))
    

print(len(words_list))
vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    # 기사 내 서로 다른 단어 개수가 5개 이상인 기사만 사용    
    if len(words) > 5:
        new_words_list.append((words, article))
        
print(len(new_words_list))

140
140


In [10]:
print(new_words_list[0][0])

['기사', '더벨', '유료', '페이지', '기사', '유니온', '투자', '파트너', '기업', '부동산', '중개', '플랫폼', '스퀘어', '투자', '토스', '테라펀딩', '프롭', '테크', '투자', '포트폴리오', '확대', '양상', '업계', '유니온', '투자', '파트너', '최근', '규모', '유니온', '프롭', '테크', '투자', '조합', '결성', '펀드', '스퀘어', '투자', '프로젝트', '펀드', '약정', '대부분', '자금', '부동산', '분야', '벤처기업', '투자', '특화', '프롭', '테크', '펀드', '지난', '출범', '약정', '유니온', '투자', '파트너', '올해', '투자', '조합', '가운데', '최대', '금액', '규모', '대표', '펀드매니저', '김영도', '이사', '핵심', '운용', '인력', '김진원', '선임', '심사', '포트폴리오', '무용', '부동산', '솔루션', '스퀘어', '이름', '오피스', '임차', '기업', '고객', '대상', '공인', '중개', '서비스', '제공', '임대료', '건물', '정보', '조사', '고객', '이상', '확보', '수요', '공급', '체계', '관리', '주목', '유니온', '투자', '파트너', '관계자', '실제', '매물', '정보', '바탕', '플랫폼', '높이', '평가', '스퀘어', '조달', '자금', '인력', '확충', '사업', '기반', '계획', '현재', '직원', '근무', '연말', '수준', '신규', '채용', '인원', '대부분', '마케팅', '부문', '배정', '스퀘어', '누적', '투자', '유치', '금액', '웃돈', '엔젤', '스벤', '파트너', '로부터', '시드', '투자', '이듬해', '진행', '시리즈', '소프트뱅크', '처스', '규모', '투자', '시리즈', '당시', '소프트뱅크', '처스', 'YJ', '캐피탈', '로부터'

## Build term-document matrix

In [11]:
#input - words_list, vocab
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd


print('기사 갯수 : ' ,len(words_list), ' ', len(new_words_list))
print('vocab 갯수 : ' ,len(vocab))
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
#for i, (words, article) in enumerate(words_list_filtering):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

기사 갯수 :  140   140
vocab 갯수 :  5485
Term document matrix shape: (140, 5485)
  (0, 5399)	0.042765780657219005
  (0, 5391)	0.021750332847031626
  (0, 5388)	0.02512454092364492
  (0, 5313)	0.02194746353432949
  (0, 5278)	0.03201865780813822
  (0, 5145)	0.10761499131817712
  (0, 5133)	0.2844355229595889
  (0, 5130)	0.042765780657219005
  (0, 5086)	0.06812727717199334
  (0, 5061)	0.01961189504579067
  (0, 5049)	0.01172894382194611
  (0, 5042)	0.04875722582124651
  (0, 5041)	0.05089566502068652
  (0, 5040)	0.10721184578675291
  (0, 5003)	0.21194492792641617
  (0, 4965)	0.05351290350629979
  (0, 4932)	0.2753796672865165
  (0, 4896)	0.22754841836767115
  (0, 4884)	0.3040808014061408
  (0, 4879)	0.061642787869767304
  (0, 4878)	0.19502890328498604
  (0, 4745)	0.0380101001757676
  (0, 4674)	0.044001548136193225
  (0, 4649)	0.029495627178802785
  (0, 4626)	0.024367572730843952
  :	:
  (139, 1234)	0.03099377057297928
  (139, 1217)	0.042334575906676615
  (139, 1180)	0.015487635064599705
  (139, 117

## Topic modeling

In [12]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
from sklearn.decomposition import NMF
 
K = 7  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (140, 7)
H shape: (7, 5485)


In [13]:
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')        
    print("\n", "="*50)

0th topic
펀드 투자 운용 결성 기업 출자 자산운용 인베스트먼트 유니온 스틱벤처스 조합 포트폴리오 설정 발굴 파트너 스틱 헤지펀드 벤처캐피탈 스퀘어 서비스 
1th topic
발행 금리 모채 회사채 두산 예측 민평 시장 건설 한화 수요 MBS BBB bp 채권 파워 조달 신용등급 AA 신용 
2th topic
상장 바이오 IPO 패스 주가 공모 기업 비나텍 밸류 청약 증권 신약 시장 프리 심사 모주 예측 닉스 몸값 코스닥 
3th topic
금융 은행 카카오 부동산 대출 클라우드 호텔 개발 롯데 신탁 BNK 상품 보험 자산 인수 서울 협의 거래 관광 증권 
4th topic
SK 하이닉스 케미칼 트론 테크놀로지 그룹 에이 조달 디스커버리 국물 금리 TSMC 보증 bp 주문 빌딩 유로본드 발행 채무 미국 
5th topic
매출 사업 생산 녹십자 영업 이익 드라마 회장 포스코 올해 상반기 공장 개선 손실 증가 실적 구조조정 LG 매각 수익 
6th topic
KCGI 인수 저축은행 스트릿 카드 스마트 BC PE 케이에스 나이스 항공 감독 LG 컨소시엄 PEF 당국 그룹 아시아나 파트너 주주 


In [14]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:5]:
        print(words_list[index][1]['title'])        
    print("\n", "="*80)

0th topic
스틱벤처스, 첫 작품 '스틱일자리펀드' 결성           780억 조달, 바이오헬스케어·ICT서비스 분야 집중
유니온투자, '문화→일반기업' 투자 축 이동           7월말 투자비중 64% AUM 3342억 확대…블라인드 펀드 결성 과제
'OEM 논란'에 자문받는 공모주펀드도 '긴장'           모호한 기준에 운용업계 '설왕설래'…DLF 감사결과 예의주시
컴퍼니케이, '유망서비스펀드' 조기 투자 소진           결성 1년반 와그트래블 등 26곳 총 830억 투자, 2020년 회수·배분 전망
KoFC-미래에셋, 신성장기업 선구안 통했다           전통제조·ICT·바이오 등 25개 기업 투자, '원금' 초과 중간회수

1th topic
금리매력 완화, 수요예측 양극화…SK로 쏠린 눈           [Market Watch]발행사 저금리 조달 제동, MBS 미매각 지속…수급 불안 관측
SK㈜, 3연타 오버부킹…시장 지위 굳건           1조 이상 수요 확인…5·10년물 오버금리 발행
한화건설, 공모채 투자수요 부진…추가 청약 기대           [Deal Story]일시적 투심 위축 vs BBB급 시장 이상기류 감지
㈜두산, 올 들어 두번째 공모채 발행           400억 규모, 증액 가능성…만기도래 회사채 차환
GS파워, 10년물 늘려 공모채 추진           내달 최대 1300억 발행…KB·NH·삼성 대표주관

2th topic
올리패스 IPO, 바이오 수요 확인…밸류 관건           [Market Watch]일반 청약 흥행…확 낮춘 몸값에 투자 수요 유입
압타바이오, 임상2상 소식도 주가 부양 '역부족'           [IPO 후 주가 점검]상장 3개월 수익률 -40% 기록…바이오 업황 악화 여파
상장사 60% 공모가 하회…프리IPO 과밸류가 문제           상장 전 과도한 밸류로 투자유치…유통시장 참패로 이어져
눈높이 낮춘 '성장성 IPO 기업', 자존심 회복하나

## Visualization

In [15]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지 (hard clustering)


W shape: (140, 7)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 140 samples in 0.000s...
[t-SNE] Computed neighbors for 140 samples in 0.003s...
[t-SNE] Computed conditional probabilities for sample 140 / 140
[t-SNE] Mean sigma: 0.115939
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.270912
[t-SNE] KL divergence after 1000 iterations: 0.203535


In [16]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [17]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})



p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

In [18]:
#topic과 Keyword 분포, 빈도를 확인할 수 있는 visualization
#jupiter note에서 사용할때 그래프 확인후 interrupt the kernel로 종료 후 다음진행 필요
from sklearn.preprocessing import normalize

def y_to_doc_topic(y):
    n_topics = y.shape[1]
    base = 1 / n_topics
    doc_topic_prob = normalize(y, norm='l1')
    rowsum = doc_topic_prob.sum(axis=1)
    doc_topic_prob[np.where(rowsum == 0)[0]] = base
    return doc_topic_prob


In [19]:
def components_to_topic_term(components):
    n_terms = components.shape[1]
    base = 1 / n_terms
    topic_term_prob = normalize(components, norm='l1')
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base
    return topic_term_prob

In [20]:
doc_topic_prob = y_to_doc_topic(W)
topic_term_prob = components_to_topic_term(H)
doc_lengths = np.asarray(term_doc_matrix.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(term_doc_matrix.sum(axis=0)).reshape(-1)

In [21]:
from pyLDAvis import prepare, show
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn


print(topic_term_prob.shape)
print(len(vocab))
prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    vocab,
    term_frequency,
    R = 20, #num of displayed terms,
    sort_topics = False
)

#display(prepared_data)
#show(prepared_data)
#pyldavis 결과 html파일로 저장
pyldavis_html_path = 'C:/Users/Administrator/Desktop/theBell_topicM.html'
pyLDAvis.save_html(prepared_data, pyldavis_html_path)

(7, 5485)
5485


C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


## Simple search engine

In [47]:
query = 'pe'
query_index = word2id[query]  # 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
#print('논제로_독', nonzero_doc);
ranks = query_vec.toarray().flatten().argsort()[::-1]
#print('랭크', ranks);



for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])


KeyError: 'pe'